In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from plotly import graph_objects as go
import plotly as py

from datetime import datetime
from datetime import timedelta

import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix, accuracy_score, f1_score, confusion_matrix, recall_score, precision_score

from sklearn.utils import resample

In [2]:
df_owm = pd.read_csv('../../../data/cleandata/Info pluviometricas/Merged Data/OpenWeatherMapSantoAndre.csv')
#df_est = pd.read_csv('../../../ordens.csv', sep=';')

In [10]:
df_owm.columns

Index(['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'temp',
       'feels_like', 'temp_min', 'temp_max', 'pressure', 'sea_level',
       'grnd_level', 'humidity', 'wind_speed', 'wind_deg', 'rain_1h',
       'rain_3h', 'snow_1h', 'snow_3h', 'clouds_all', 'weather_id',
       'weather_main', 'weather_description', 'weather_icon'],
      dtype='object')

In [5]:
df_owm['wind_speed']

0         1.34
1         1.43
2         1.66
3         1.02
4         1.30
          ... 
370034    4.10
370035    3.60
370036    5.10
370037    3.60
370038    4.10
Name: wind_speed, Length: 370039, dtype: float64

In [177]:
df_owm['Data_Hora'] = pd.to_datetime(df_owm['dt_iso'].str[:-10])
df_owm['Data_Hora'] = df_owm.apply(lambda x: x['Data_Hora'] + pd.Timedelta(hours = x['timezone'] / 3600), axis = 1)
df_owm = df_owm[(datetime.strptime('2019-08-30', '%Y-%m-%d') >= df_owm['Data_Hora']) & (df_owm['Data_Hora'] >= datetime.strptime('2010-01-01', '%Y-%m-%d'))]
df_owm = df_owm.drop(columns = ['sea_level', 'grnd_level', 'rain_3h', 'snow_1h', 'snow_3h'])
df_owm = df_owm.fillna(0)
df_owm = df_owm.drop_duplicates(subset='Data_Hora')

In [178]:
df_est['Data'] = pd.to_datetime(df_est['Data'], yearfirst=True)
df_est.head()

,Data,Vitoria,Erasmo,Paraiso,Null,RM,Camilopolis
0,2010-01-07,21,4,0,0,0,0
1,2010-01-08,11,0,0,0,0,0
2,2010-01-11,1,1,0,0,0,0
3,2010-01-19,3,0,2,1,0,0
4,2010-01-20,11,0,0,1,0,0


In [1]:
df_loc = pd.read_csv('../../../data/cleandata/Ordens de serviço/labels_day.csv', sep=';')
df_loc['Data'] = pd.to_datetime(df_loc['Data'], yearfirst=True)
df_loc = df_loc[['Data', 'LocalMax']]
df_loc.columns = ['Data', 'Label']
df_loc.head()

NameError: name 'pd' is not defined

In [180]:
print(df_est['Data'].min())
print(df_est['Data'].max())

2010-01-07 00:00:00
2019-08-30 00:00:00


In [181]:
# df_est = pd.read_csv('../../../data/cleandata/Ordens de serviço/Ordens_Label.csv', sep = ';')
# df_est['Data'] = pd.to_datetime(df_est['Data'], yearfirst=True)
# df_est = df_est[['Data', 'LocalMax']]
# df_est.columns = ['Data', 'Label']
# df_owm['Data'] = pd.to_datetime(df_owm['Data_Hora'].dt.strftime('%Y-%m-%d'), yearfirst=True)
# df = df_owm.merge(df_est, on='Data', how='left')
# df = df.fillna(0)

In [182]:
df_owm['Data'] = pd.to_datetime(df_owm['Data_Hora'].dt.strftime('%Y-%m-%d'), yearfirst=True)
df = df_owm.merge(df_loc, on='Data', how='left')
df = df.fillna(0)
#df['Label'] = df.apply(lambda x: 1 if x['Vitoria'] + x['Erasmo'] + x['Paraiso'] + x['Null'] + x['RM'] + x['Camilopolis'] > 0 else 0, axis = 1)

In [183]:
df_g = df.groupby('Data').sum().reset_index()[['Data', 'rain_1h']]
df_g.columns = ['Data', 'rain_sum']
df = df.merge(df_g, on='Data')

In [184]:
df['Mes'] = df['Data_Hora'].dt.month
df['Hora'] = df['Data_Hora'].dt.hour
df['Dia'] = df['Data_Hora'].dt.day
# df = df.drop(columns = ['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'weather_icon', 'weather_id', 'weather_main',
#                         'Vitoria', 'Erasmo', 'Paraiso', 'RM', 'Null', 'Camilopolis'])
df = df.drop(columns = ['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'weather_icon', 'weather_id', 'weather_main'])
# df = df.drop(columns = ['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'weather_icon', 'weather_id', 'weather_main',
#                         'Data_Hora', 'Data'])
df['weather_description'] = df['weather_description'].rank(method='dense', ascending=False).astype(int)

In [185]:
#df = df.groupby('Data').max()

In [186]:
df_slice = df.copy()
#df_slice = df_clustered_total[(df_clustered_total['Cluster'].isin([0]))]
#df_slice.loc[df_slice['Cluster'] == 0, 'Ordens'] = 0
df_slice.loc[(df_slice['Label'] == 1) & (df_slice['rain_sum'] <= 10), 'Label'] = 0

In [187]:
df_slice.groupby('Label').count()

,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,clouds_all,weather_description,Data_Hora,Data,rain_sum,Mes,Hora,Dia
Label,,,,,,,,,,,,,,,,,
0.0,84064,84064,84064,84064,84064,84064,84064,84064,84064,84064,84064,84064,84064,84064,84064,84064,84064
1.0,600,600,600,600,600,600,600,600,600,600,600,600,600,600,600,600,600


In [188]:
xgb = xgboost.XGBClassifier()

cols_rem = ['Label', 'Data', 'Data_Hora'] #+ ['temp_min', 'temp_max', 'clouds_all', 'weather_description']
cols_rem = cols_rem + ['temp', 'feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'clouds_all', 'weather_description']

x = df_slice[[c for c in df_slice.columns if c not in cols_rem]]
print(x.columns)
#x = df_slice.drop(columns = 'Label')
#x = x.drop(columns = 'Cluster')
y = df_slice['Label']

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state = 378, stratify=y)

# concatenate our training data back together
X = pd.concat([x_treino, y_treino], axis=1)

# separate minority and majority classes
not_ordem = X[X['Label']==0].copy()
ordem = X[X['Label']==1].copy()

# upsample minority
ordem_upsampled = resample(ordem,
                        replace=True, # sample with replacement
                        n_samples=len(not_ordem), # match number in majority class
                        random_state=378) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_ordem, ordem_upsampled])

x_treino = upsampled.drop(columns = 'Label')
y_treino = upsampled['Label']

display(y_treino.value_counts())

#xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino), (x_teste, y_teste)], eval_metric=f1_score)
param = {'max_depth':50, 'eta':1, 'objective':'binary:logistic', 'min_child_weight': 1, 'lambda': 1, 'alpha': 0, 'gamma': 0}

df_train = xgboost.DMatrix(data=x_treino, label=y_treino)
df_test = xgboost.DMatrix(data=x_teste, label=y_teste)

bst = xgboost.train(param, df_train, 2, feval=f1_score)
y_teste_pred = bst.predict(xgboost.DMatrix(data=x_teste, label=y_teste))
y_teste_pred = [1 if i>0.5 else 0 for i in y_teste_pred]
y_treino_pred = bst.predict(xgboost.DMatrix(data=x_treino, label=y_treino))
y_treino_pred = [1 if i>0.5 else 0 for i in y_treino_pred]

print(f"Treino: {accuracy_score(y_treino, y_treino_pred)}")
print(f"Teste: {accuracy_score(y_teste, y_teste_pred)}")
print(f"Precisão: {precision_score(y_teste, y_teste_pred)}")
print(f"Recall: {recall_score(y_teste, y_teste_pred)}")
print(f"F1: {f1_score(y_teste, y_teste_pred)}")
display(confusion_matrix(y_teste, y_teste_pred, normalize='true'))
display(confusion_matrix(y_teste, y_teste_pred,))

Index(['rain_1h', 'rain_sum', 'Mes', 'Hora', 'Dia'], dtype='object')


1.0    58844
0.0    58844
Name: Label, dtype: int64

Treino: 0.9997195975800421
Teste: 0.9992913385826772
Precisão: 0.9090909090909091
Recall: 1.0
F1: 0.9523809523809523


array([[9.99286281e-01, 7.13719270e-04],
       [0.00000000e+00, 1.00000000e+00]])

array([[25202,    18],
       [    0,   180]], dtype=int64)

In [189]:
x_treino

,rain_1h,rain_sum,Mes,Hora,Dia
36736,0.00,2.36,3,20,11
56833,0.00,0.00,6,7,26
885,0.00,2.56,2,21,6
17618,0.00,0.90,1,4,5
62946,0.00,0.00,3,1,8
...,...,...,...,...,...
72033,0.00,10.76,3,17,21
8322,2.86,14.79,12,19,13
18134,0.72,23.61,1,16,26
83317,2.21,49.72,7,22,4


In [190]:
x_a = x_teste.copy()
x_a['Label'] = y_teste
x_a['Label_Pred'] = y_teste_pred
x_a[x_a['Label_Pred'] == 1]

,rain_1h,rain_sum,Mes,Hora,Dia,Label,Label_Pred
2300,0.00,25.18,4,20,6,0.0,1
53661,0.00,11.01,2,3,15,1.0,1
80002,0.17,28.31,2,19,16,1.0,1
27920,0.00,12.27,3,11,9,1.0,1
61098,2.00,12.71,12,1,21,1.0,1
...,...,...,...,...,...,...,...
15668,0.71,38.32,10,21,15,1.0,1
36104,0.00,12.50,2,12,13,1.0,1
79994,2.52,28.31,2,11,16,1.0,1
9531,0.00,24.43,2,4,2,1.0,1


In [191]:
def getPrecMomento(x):
    return df_label_pred.loc[
        (df_label_pred['Data'] == x['Data']) &
        (df_label_pred['Data_Hora'] <= x['Data_Hora']), 'rain_1h'
    ].sum()

#df_cluster_hora['PrecMomento'] = df_cluster_hora.apply(getPrecMomento, axis=1)
#df_cluster_hora.head(100).apply(getPrecMomento, axis=1)


df_pred = bst.predict(xgboost.DMatrix(data=df_slice[['rain_1h', 'rain_sum', 'Mes', 'Hora', 'Dia']]))
df_pred = [1 if i>0.5 else 0 for i in df_pred]
df_slice['Label_Pred'] = df_pred
#df_slice.loc[df_slice['Label_Pred'] == 1, ['Data_Hora', 'rain_1h', 'rain_sum', 'Mes', 'Label', 'Label_Pred']]

df_label_pred = df_slice.loc[df_slice['Label_Pred'] == 1, ['Data_Hora', 'Data', 'Dia', 'Hora', 'rain_1h', 'rain_sum', 'Mes', 'Label', 'Label_Pred']].copy()
df_label_pred['rain_momento'] = df_label_pred.apply(getPrecMomento, axis=1)
df_label_pred.head(24)

df_pred = bst.predict(xgboost.DMatrix(data=df_label_pred[['rain_1h', 'rain_momento', 'Mes', 'Hora', 'Dia']].rename(columns={'rain_momento':'rain_sum'})))
df_pred = [1 if i>0.5 else 0 for i in df_pred]

df_label_pred['Label_Momento'] = df_pred

In [195]:
df_label_pred[['Data_Hora', 'rain_1h', 'rain_momento', 'rain_sum', 'Label', 'Label_Pred', 'Label_Momento']]

,Data_Hora,rain_1h,rain_momento,rain_sum,Label,Label_Pred,Label_Momento
144,2010-01-07 00:00:00,0.00,0.00,14.75,1.0,1,0
145,2010-01-07 01:00:00,0.00,0.00,14.75,1.0,1,0
146,2010-01-07 02:00:00,0.00,0.00,14.75,1.0,1,0
147,2010-01-07 03:00:00,0.00,0.00,14.75,1.0,1,0
148,2010-01-07 04:00:00,0.00,0.00,14.75,1.0,1,0
...,...,...,...,...,...,...,...
83314,2019-07-04 19:00:00,2.53,40.08,49.72,1.0,1,1
83315,2019-07-04 20:00:00,2.18,42.26,49.72,1.0,1,1
83316,2019-07-04 21:00:00,2.38,44.64,49.72,1.0,1,1
83317,2019-07-04 22:00:00,2.21,46.85,49.72,1.0,1,1


In [196]:
df_label_pred[['Data_Hora', 'rain_1h', 'rain_momento', 'rain_sum', 'Label_Pred', 'Label_Momento']].to_csv("labels_prediction.csv", index=False, sep=";", decimal=",")